# Pre-procesamiento en Python

Data Quality & Drifting sobre periódos de:

- 6 meses

IAN repo como base

In [ ]:
# Librerias

import pandas as pd

###########
# funciones

from utils import psi, drift_deflacion

#######
# rutas
from config import dataset_file_crudo, \
                   dataset_file_clase_ternaria, \
                   dataset_file_prepro_6#, dataset_file_prepro_9

##############
# finance info
from finance import tb_indices

Variables de train y test

In [ ]:
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train_all = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
                 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
                 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]

mes_train_ult_6_meses = [202101, 202102, 202103, 202104, 202105, 202106]

mes_train_ult_9_meses = [202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106]

mes_train_ult_anio = [202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]

# mes_train = 202106
mes_test = 202108

semillas = [211777, 174989, 131497, 612223, 234803]

## Loading data

In [ ]:
data = pd.read_csv(dataset_file_clase_ternaria)

# preprocesando 6 meses

train_data = data[data['foto_mes'].isin(mes_train_ult_6_meses)]
score_data = data[data['foto_mes'] == mes_test]

## A. Data Quality

Nulos

In [ ]:
train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted.head(5)

Variables con gran porcentaje de nulos

In [ ]:
comparison_df_sorted.loc[comparison_df_sorted["diff"]> 25]

Sin correcciones hechas con nulos

In [ ]:
###

Valores 0

In [ ]:
train_zero_percentage = (train_data == 0).mean() * 100
score_zero_percentage = (score_data == 0).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Zero Percentage': train_zero_percentage, 'Score Zero Percentage': score_zero_percentage})

comparison_df_zero['diff_zero_percentage'] = (comparison_df_zero['Score Zero Percentage'] - comparison_df_zero['Train Zero Percentage']).abs()
diff_zero_percentage_sorted = comparison_df_zero.sort_values('diff_zero_percentage',ascending=False)

diff_zero_percentage_sorted.head(5)

Variables con diferencia severa de ceros

In [ ]:
diff_zero_percentage_sorted.loc[diff_zero_percentage_sorted["diff_zero_percentage"] > 25]

Limpieza

In [ ]:
cols_eliminar = list(diff_zero_percentage_sorted.loc[diff_zero_percentage_sorted["diff_zero_percentage"] > 25].index())

# cols_eliminar = list(diff_zero_percentage_sorted[\
#     (diff_zero_percentage_sorted["diff_zero_percentage"] > 5)&\
#         (diff_zero_percentage_sorted["Score Zero Percentage"]>diff_zero_percentage_sorted["Train Zero Percentage"])].index())

data.drop(cols_eliminar,axis=1,inplace=True)

train_data.drop(cols_eliminar,axis=1,inplace=True)
score_data.drop(cols_eliminar,axis=1,inplace=True)

Negativos y Duplicados no son requeridos en la limpieza, según EDA

## B. Data Drifting

In [ ]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes','numero_de_cliente','clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)

Variables con DF

In [ ]:
psi_df[psi_df["psi"] > 0.1]["feature"]

Limpieza

In [ ]:
cols_eliminar = list(psi_df[psi_df["psi"] > 0.1]["feature"])

data.drop(cols_eliminar,axis=1,inplace=True)

train_data.drop(cols_eliminar,axis=1,inplace=True)
score_data.drop(cols_eliminar,axis=1,inplace=True)

## D. Ajustes por inflación

In [ ]:
campos_monetarios = [col for col in data.columns if col.startswith(('m', 'Visa_m', 'Master_m', 'vm_m'))]

In [ ]:
data = drift_deflacion(data, campos_monetarios, tb_indices)

## Exportando data prepro

In [ ]:
data.to_csv(dataset_file_prepro_6,index=False)